In [1]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


이 노트북은 다음 주소에 있는 노트북을 사용하는데 parameter값이나, 데이터셋 사용에서 약간의 변형을 하였다:
원 주소: https://metamath1.github.io/blog/posts/gentle-t5-trans/gentle_t5_trans.html

google colab resource 사용량

cpu = 4.8/12.7GB
gpu = 13.0/15.0GB

In [2]:
# sentencepiece is required for t5 type models
!pip install sentencepiece
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.6 MB/s eta 0:00:00


In [1]:
!pip install transformers datasets evaluate rouge_score sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=6ba6dc88bdd182561d0ecf379e62500d2f0f99748d9dc866051507e356f

In [2]:
! pip install huggingface_hub

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# Original data size is 3 times larger than 400000, i.e. 1200000
# Here, only the train split is chosen, for the sake of simplicity

from datasets import load_dataset
raw_ds = load_dataset("bongsoo/news_talk_en_ko",split="train").select(range(400000))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset = raw_ds.train_test_split(train_size=0.9,seed=20)
dataset

DatasetDict({
    train: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 360000
    })
    test: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 40000
    })
})

In [6]:
dataset["validation"] = dataset.pop("test")

In [7]:
dataset = dataset.rename_column("Skinner's reward is mostly eye-watering.",'en')
dataset = dataset.rename_column("스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.",'ko')

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 360000
    })
    validation: Dataset({
        features: ['en', 'ko'],
        num_rows: 40000
    })
})

In [9]:
# Lines From 9 to 13 are not necessary and need not be executed.
import pandas as pd
dataset.set_format(type="pandas")

In [10]:
df = dataset["train"][:]

In [11]:
df.head()

,en,ko
0,Aside from the discussion on Geumgangsan and K...,"금강산·개성을 논외로 하면, 유엔 결의 2397호가 규정한 ‘한해 정제유 50만t’..."
1,The price of farmland skyrocketed as various i...,양산시에 각종 산업단지가 대거 들어서면서 농지 값이 급등했다.
2,Relief is that as the will of the government o...,다행스러운 것은 혁신 성장에 정부 의지가 반영돼 내년도 정부 연구개발(R&D) 예산...
3,They say that the science and technology ecosy...,연구인력 수급난으로 과학기술 생태계가 무너지고 사기가 저하되고 있다는 것이다.
4,"However, as the news of the bankruptcy of Lehn...",다만 지난 2008년 추석연휴에 미국의 리먼 브라더스 파산 소식이 전해지면서 개장과...


In [12]:
dataset.reset_format()

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 360000
    })
    validation: Dataset({
        features: ['en', 'ko'],
        num_rows: 40000
    })
})

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

다음은 데이터 콜레이터를 임포트한다. 시쿼스 투 시퀀스 학습 과정은 인코더 입력 시퀀스, 디코더 입력 시퀀스, 디코더 출력 시퀀스를 필요로 하는데 미니 배치로 부터 이를 적절히 정리해서 모델에 입력하는 작업이 필요하다. 예를 들면 미니 배치 내에 있는 인코더 입력 시퀀스의 길이를 맞춘다든지 디코더 입력시퀀스를 오른쪽으로 한 칸 쉬프트시켜 디코더 출력 시퀀스를 만드는 작업 등이 콜레이터에서 일어나는 작업인데 이런 작업을 DataCollatorForSeq2Seq가 자동으로 처리하게 된다.

In [15]:
from transformers import DataCollatorForSeq2Seq

In [16]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [17]:
from datasets import load_dataset, load_metric

In [18]:
import numpy as np
import torch
import multiprocessing

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [20]:
# 한국 전자기술 연구원 T5
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [22]:
dataset['train'][10]['en'], dataset['train'][10]['ko']

("Prosecutors have secured testimony that Mok was the leading figure in the advisory group's meeting and that Choi, a former executive vice president of Samsung Electronics Services who was arrested and indicted, attended the meeting.",
 '검찰은 목 전 전무가 이 자문단 회의를 주도한 인물이라고 보고 있으며, 이 자리에 구속기소된 최아무개 전 삼성전자서비스 전무 등이 참석했다는 진술을 확보했다.')

In [23]:
tokenized_sample_en = tokenizer(dataset['train'][10]['en'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_en

{'input_ids': [51032, 84, 25796, 24626, 38, 3171, 1033, 62, 5, 3426, 6413, 20, 5, 32645, 1113, 17, 8, 2228, 13, 38, 17350, 644, 4, 16, 1248, 3465, 8211, 1296, 14, 13005, 33592, 8, 7224, 135, 62, 5741, 13, 52588, 4, 11282, 5, 2228, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
tokenized_sample_ko = tokenizer(dataset['train'][10]['ko'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_ko

{'input_ids': [2084, 2545, 75, 9182, 22, 47, 7, 46527, 5811, 18201, 4853, 115, 444, 773, 4, 47, 4252, 23200, 178, 511, 34623, 75, 2341, 1910, 9182, 410, 3741, 12, 14209, 11346, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [25]:
tokenizer(dataset['train'][:3]['en'],
          max_length=max_token_length,
          padding=True, truncation=True)

{'input_ids': [[53640, 73, 5, 9875, 46, 10009, 3793, 694, 5117, 22069, 13, 7204, 566, 5097, 4, 5, 414, 3, 1221, 3, 62643, 573, 5584, 28155, 5, 17788, 9816, 14, 77, 10063, 26990, 14, 59711, 2629, 89, 46536, 54292, 77, 18, 117, 48226, 8, 322, 2825, 14, 843, 22299, 73, 13305, 3, 1], [55, 2171, 14, 11058, 2703, 62658, 102, 64, 4395, 11110, 6854, 566, 11689, 5, 53150, 22069, 28, 6523, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [62119, 43, 38, 64, 5, 96, 14, 5, 618, 46, 17612, 2048, 43, 33121, 4, 5, 7054, 13, 8435, 26, 1127, 413, 492, 18, 3452, 14, 5, 618, 96, 34385, 195, 18885, 1234, 807, 256, 4, 40, 5, 275, 248, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

미니배치에 있는 샘플의 최대길이메 맞춰서 패딩되는 모습을 확인할 수 있다. 실제로 어떻게 토큰화 되었는지 확인해보자.

In [26]:
import pandas as pd
pd.DataFrame(
    [
        tokenized_sample_en['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_en['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
ids,51032,84,25796,24626,38,3171,1033,62,5,3426,...,62,5741,13,52588,4,11282,5,2228,3,1
tokens,▁Prosecutors,▁have,▁secured,▁testimony,▁that,▁Mo,k,▁was,▁the,▁leading,...,▁was,▁arrested,▁and,▁indicted,",",▁attended,▁the,▁meeting,.,</s>


마지막에 엔드 토큰인 </s>가 항상 붙게 되는 것도 확인할 수 있다.

In [27]:
pd.DataFrame(
    [
        tokenized_sample_ko['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_ko['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
ids,2084,2545,75,9182,22,47,7,46527,5811,18201,...,2341,1910,9182,410,3741,12,14209,11346,3,1
tokens,▁검찰은,▁목,▁전,▁전무,가,▁이,▁,자문단,▁회의를,▁주도한,...,▁삼성전자,서비스,▁전무,▁등이,▁참석했다,는,▁진술을,▁확보했다,.,</s>


In [28]:
def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['en'],
                             text_target=examples['ko'],
                             max_length=max_token_length, truncation=True)

    return model_inputs

기본으로 미니 배치 사이즈는 1000으로 세팅되어 있다.

미니 배치로 넘어온 문장 샘플을 영어 문장과 한국어 문장을 각각 인풋과 타겟으로 토큰화하고 이로 부터 input_ids, attention_mask, labels로 묶어 리턴하는 방식이 예전에 쓰던 방식으로 함수 위쪽에 주석처리 되어 있다. 타겟 문장을 토큰화 할 때 타겟에서 필요로 하는 특수 토큰을 추가하는 경우 이를 처리하기위해 타겟 토큰 토큰화 때는 with tokenizer.as_target_tokenizer():라는 컨텍스트 매니저를 사용했는데 최근 업데이트에서는 그냥 tokenizer에 text_target인자에 타겟 문장을 넣어서 한번에 다 처리할 수 있다. 이렇게 model_inputs을 반환하면 dataset에 있던 각 레코드 마다 en, ko 특성에 추가로 input_ids, attention_mask, labels 특성이 더 추가 되게 된다. 사실 en, ko 특성은 더이상 필요없기 때문에 convert_examples_to_features()를 적용할 때 없애라는 인자를 세팅한다.

In [29]:
NUM_CPU = multiprocessing.cpu_count()
NUM_CPU

2

그리고 remove_columns 인자에 기존 특성 이름인 en, ko를 전달해서 기존 특성은 제거하게 한다. 이 특성이 있으면 이후 콜레이터가 샘플들을 미니 배치로 묶을 때 패딩처리를 못하게 된다.

In [30]:
tokenized_datasets = dataset.map(convert_examples_to_features,
                                 batched=True,
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서
                                 # 아래서 콜레이터가 패딩을 못해서 에러남
                                 remove_columns=dataset["train"].column_names,
                                 num_proc=NUM_CPU)
# tokenized_datasets = dataset.map(convert_examples_to_features,
#                                  batched=True,
#                                  # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서
#                                  # 아래서 콜레이터가 패딩을 못해서 에러남
#                                  remove_columns=dataset["train"].column_names)

Map (num_proc=2):   0%|          | 0/360000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/40000 [00:00<?, ? examples/s]

In [31]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 360000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 40000
    })
})

In [32]:
tokenized_datasets['train'][10]

{'input_ids': [51032,
  84,
  25796,
  24626,
  38,
  3171,
  1033,
  62,
  5,
  3426,
  6413,
  20,
  5,
  32645,
  1113,
  17,
  8,
  2228,
  13,
  38,
  17350,
  644,
  4,
  16,
  1248,
  3465,
  8211,
  1296,
  14,
  13005,
  33592,
  8,
  7224,
  135,
  62,
  5741,
  13,
  52588,
  4,
  11282,
  5,
  2228,
  3,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [2084,
  2545,
  75,
  9182,
  22,
  47,
  7,
  46527,
  5811,
  18201,
  4853,
  115,
  444,
  773,
  4,
  47,
  4252,
  23200,
  178,
  511,
  34623,
  75,
  2341,
  1910,
  9182,
  410,
  3741,
  12,
  14209,
  11346,
  3,
  1]}

In [33]:
print( '원 데이터    :', dataset['train'][10]['en'] )
print( '처리 후 데이터:', tokenized_datasets['train'][10]['input_ids'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['input_ids']) )

print('\n')
print( '원 데이터    :', dataset['train'][10]['ko'] )
print( '처리 후 데이터:', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['labels']) )
print( '토큰화       :', tokenized_datasets['train'][10]['labels'] )

원 데이터    : Prosecutors have secured testimony that Mok was the leading figure in the advisory group's meeting and that Choi, a former executive vice president of Samsung Electronics Services who was arrested and indicted, attended the meeting.
처리 후 데이터: [51032, 84, 25796, 24626, 38, 3171, 1033, 62, 5, 3426, 6413, 20, 5, 32645, 1113, 17, 8, 2228, 13, 38, 17350, 644, 4, 16, 1248, 3465, 8211, 1296, 14, 13005, 33592, 8, 7224, 135, 62, 5741, 13, 52588, 4, 11282, 5, 2228, 3, 1]
토큰화       : ['▁Prosecutors', '▁have', '▁secured', '▁testimony', '▁that', '▁Mo', 'k', '▁was', '▁the', '▁leading', '▁figure', '▁in', '▁the', '▁advisory', '▁group', "'", 's', '▁meeting', '▁and', '▁that', '▁Cho', 'i', ',', '▁a', '▁former', '▁executive', '▁vice', '▁president', '▁of', '▁Samsung', '▁Electronic', 's', '▁Services', '▁who', '▁was', '▁arrested', '▁and', '▁indicted', ',', '▁attended', '▁the', '▁meeting', '.', '</s>']


원 데이터    : 검찰은 목 전 전무가 이 자문단 회의를 주도한 인물이라고 보고 있으며, 이 자리에 구속기소된 최아무개 전 삼성전자서비스 전무 등이 참석했다는 진술을 확

In [34]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
model

T5ForConditionalGeneration(
  (shared): Embedding(64128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

위처럼 모델을 로딩하고 모델 출력 시켜보면 T5 모델 레이어가 매우 길게 출력되는데 제일 마지막 부분에 다음과 같이 분류 헤드가 붙어 있는 것을 확인할 수 있다. 헤드를 보면 모델에서 출력하는 벡터는 768차원이고 이를 단어장 사이즈인 64128로 변환시키고 있는 것을 알 수 있다.

(lm_head): Linear(in_features=768, out_features=64128, bias=False)
이렇게 생성된 model은 인코더-디코더 구조를 가지는 트랜스포머이므로 이 모델을 포워딩 하려면 인코더 인풋과 디코더 인풋을 넣어줘야 한다. 모델을 만들고 가장 먼저해야되는 작업은 포워딩 테스트라고 개인적으로 생각한다. 임의의 입력을 넣고 출력이 의도대로 나오는지 확인하는 것이다. 이런 작업은 직접 만든 모델이 아닐 수록 중요한데 이렇게 해야지 모델이 제대로 작동하는지 또 어떤 구조로 되어 있는지 쉽게 이해할 수 있기 때문이다. 포워드 테스트를 하기위해 간단한 영어문장으로 예제를 준비한다.

In [35]:
encoder_inputs = tokenizer(
    ["Studies have been shown that owning a dog is good for you"],
    return_tensors="pt"
)['input_ids'].to(device)

decoder_targets = tokenizer(
    ["개를 키우는 것이 건강에 좋다는 연구 결과가 있습니다."],
    return_tensors="pt"
)['input_ids'].to(device)

In [36]:
print( encoder_inputs )
print( decoder_targets )

tensor([[24611,    84,   166,  8135,    38,   847,    91,    16,  8146,    43,
           667,    40,   106,     1]], device='cuda:0')
tensor([[15833, 12236,   179, 16120, 28117,  1007,  3883,   327,     3,     1]],
       device='cuda:0')


이제 디코더 입력을 만들기위해 model._shift_right를 사용해 디코더 출력을 오른쪽으로 쉬프트 시킨다.

In [37]:
decoder_inputs = model._shift_right(decoder_targets)

decoder_inputs와 decoder_targets이 어떻게 다른지 비교해보면

In [38]:
pd.DataFrame(
    [
        tokenizer.convert_ids_to_tokens(decoder_targets[0]),
        tokenizer.convert_ids_to_tokens(decoder_inputs[0])
    ],
    index=('decoder target', 'decoder input')
)

,0,1,2,3,4,5,6,7,8,9
decoder target,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.,</s>
decoder input,<pad>,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.


위처럼 오른쪽으로 쉬프트된 디코더 입력은 <pad> 토큰이 추가되었다. 이렇게 출력으로 쓰이는 문장을 오른쪽으로 쉬프트시켜 티처포싱Teacher forcing을 진행하게 된다. 다음처럼 model에 인코더 입력, 디코더 입력, 디코더 타겟을 입력하고 포워드 시킨다.

In [39]:
# forward pass
outputs = model(input_ids=encoder_inputs,
                decoder_input_ids=decoder_inputs,
                labels=decoder_targets)

model의 outputs에는 다음과 같은 키가 있다.

In [40]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

손실함수 값을 다음처럼 확인할 수 있고 grad_fn이 있기 때문에 output.loss를 백워드 시킬 수 있다.

In [41]:
outputs.loss

tensor(87.8185, device='cuda:0', grad_fn=<NllLossBackward0>)

인코더의 마지막 상태는 (1, 14, 768)이다. 각 숫자는 순서대로 샘플 수, 스탭 수, 모델 사이즈를 나타낸다. 즉 인코더로 들어가는 14개 토큰이 각각 768차원 벡터로 인코딩되었다.

In [42]:
outputs['encoder_last_hidden_state'].shape

torch.Size([1, 14, 768])

In [43]:
outputs['logits'].shape

torch.Size([1, 10, 64128])

logit에 argmax를 씌워서 토큰화시켜보면 다음과 같다.

In [44]:
tokenizer.convert_ids_to_tokens( torch.argmax(outputs['logits'][0], axis=1).cpu().numpy() )

['큐브', '큐브', '▁비일비재', '▁비일비재', '▁베네', '▁비일비재', '▁베네', '▁베네', '큐브', '큐브']

마지막 헤더가 학습이 되지 않았기 때문에 적절한 아웃풋이 나오지 않지만 입력과 출력의 텐서 모양을 보면 포워드 패스가 제대로 작동한다는 것을 알 수 있다.

지금까지 데이터 셋, 토크나이저, 모델에 대해서 알아봤다. 이제 학습을 위해 두 단계가 남았는데 하나는 데이터를 미니배치 형태로 모아 주는 콜레이터collator와 나머지 하나는 모델을 평가할 매트릭이다

['큐브', '큐브', '▁비일비재', '▁비일비재', '▁베네', '▁비일비재', '▁베네', '▁베네', '큐브', '큐브']
마지막 헤더가 학습이 되지 않았기 때문에 적절한 아웃풋이 나오지 않지만 입력과 출력의 텐서 모양을 보면 포워드 패스가 제대로 작동한다는 것을 알 수 있다.

지금까지 데이터 셋, 토크나이저, 모델에 대해서 알아봤다. 이제 학습을 위해 두 단계가 남았는데 하나는 데이터를 미니배치 형태로 모아 주는 콜레이터collator와 나머지 하나는 모델을 평가할 매트릭이다

Collator
파이토치에서 모델을 학습시키기 위해서 DataLoader를 사용하게 되는데 이 데이터 로더의 역할은 for 루프를 돌면서 데이터 셋으로 부터 샘플을 미니 배치 수만큼 가져오는 것이다. 이때 샘플을 미니 배치 수만큼 무작위로 가져와 어떤 식으로든 각 샘플을 짝맞춤해서 반환해야하는데 크기가 통일된 간단한 이미지 데이터인 경우 특별히 할것이 없지만 서로 크기가 다른 샘플들을 다루는 경우는 반환전 크기 또는 길이를 맞춘다든지 패딩을 한다든지 하는 추가 작업이 필요하게 된다. 이런 작업이 일어나는 곳이 collate_fn으로 지정되는 함수이다.

시퀀스 투 시퀀스 모델을 학습시킬때 이런 콜레이터 함수가 하는 전형적인 역할은 입력 또는 출력 문자열을 패딩하고 조금 전 모델에서 알아봤듯이 디코더 타겟을 오른쪽으로 한칸 쉬프트 시켜서 디코더 입력으로 만드는 일이다. 앞서 이런 과정을 간단히기 직접 코딩해서 확인했지만 이런 작업을 자동으로 처리해주는 클래스가 DataCollatorForSeq2Seq이다.

우선 콜레이터를 만들기 위해서는 토크나이저와 모델을 넘겨야 한다.

In [45]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

앞서 만들어논 tokenized_datasets에서 샘플 두개를 조회하면 다음처럼 전체 결과는 사전으로 리턴되며 사전의 각 키 아래에 여러 샘플들의 값이 리스트로 들어있게 된다.

In [46]:
# 각 항목아래 샘플들이 리스트 형태로 묶여 반환된다.
tokenized_datasets["train"][1:3]

{'input_ids': [[55,
   2171,
   14,
   11058,
   2703,
   62658,
   102,
   64,
   4395,
   11110,
   6854,
   566,
   11689,
   5,
   53150,
   22069,
   28,
   6523,
   3,
   1],
  [62119,
   43,
   38,
   64,
   5,
   96,
   14,
   5,
   618,
   46,
   17612,
   2048,
   43,
   33121,
   4,
   5,
   7054,
   13,
   8435,
   26,
   1127,
   413,
   492,
   18,
   3452,
   14,
   5,
   618,
   96,
   34385,
   195,
   18885,
   1234,
   807,
   256,
   4,
   40,
   5,
   275,
   248,
   3,
   1]],
 'attention_mask': [[1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1]],
 'labels': [[50798,
   9,
   1053,
   14585,
   22,
   5247,
   47502,
   29342,
   29037,
   31032,
  

콜레이터에 샘플을 넘길 때는 개별 샘플이 사전으로 묶이는 형태가 되어야 되므로 아래처럼 한번 가공하게 된다.

In [47]:
# 콜레이터에는 샘플을 개별 {}로 넘겨야 됨
[tokenized_datasets["train"][i] for i in range(1, 3)]

[{'input_ids': [55,
   2171,
   14,
   11058,
   2703,
   62658,
   102,
   64,
   4395,
   11110,
   6854,
   566,
   11689,
   5,
   53150,
   22069,
   28,
   6523,
   3,
   1],
  'attention_mask': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  'labels': [50798,
   9,
   1053,
   14585,
   22,
   5247,
   47502,
   29342,
   29037,
   31032,
   3,
   1]},
 {'input_ids': [62119,
   43,
   38,
   64,
   5,
   96,
   14,
   5,
   618,
   46,
   17612,
   2048,
   43,
   33121,
   4,
   5,
   7054,
   13,
   8435,
   26,
   1127,
   413,
   492,
   18,
   3452,
   14,
   5,
   618,
   96,
   34385,
   195,
   18885,
   1234,
   807,
   256,
   4,
   40,
   5,
   275,
   248,
   3,
   1],
  'attention_mask': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
  

위에 반환된 결과를 보면 각 샘플이 사전 {}으로 묶이고 샘플 하나에는 input_ids, attention_mask, labels이 존재한다. 각 샘플을 리스트로 묶어서 콜레이터에게 전달하고 반환되는 값을 확인해보자.

In [48]:
# 콜레이터를 돌리면 알아서 패딩하고 쉬프트 시킨다.
batch = data_collator(
    [tokenized_datasets["train"][i] for i in range(1, 3)]
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


반환된 batch의 키를 확인해보면 decoder_input_ids가 생긴것을 확인할 수 있다.

In [49]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [50]:
batch

{'input_ids': tensor([[   55,  2171,    14, 11058,  2703, 62658,   102,    64,  4395, 11110,
          6854,   566, 11689,     5, 53150, 22069,    28,  6523,     3,     1,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [62119,    43,    38,    64,     5,    96,    14,     5,   618,    46,
         17612,  2048,    43, 33121,     4,     5,  7054,    13,  8435,    26,
          1127,   413,   492,    18,  3452,    14,     5,   618,    96, 34385,
           195, 18885,  1234,   807,   256,     4,    40,     5,   275,   248,
             3,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

새로 생긴 decoder_input_ids는 앞에 0()이 붙어 있는 것이 보이고 label에서 끝에 1이 사라져 label이 오른쪽으로 쉬프트된 것임을 알 수 있다. 그리고 또 두번째 샘플 labels에서 마지막에 -100 이 보인다. 이 값은 label이 패딩된 것을 나타내며 손실 함수값을 계산할 때 -100이 있는 위치는 손실을 계산하지 않게 된다. 이렇게 시퀀스 투 시퀀스 모델을 학습하기 위해 필요한 자잘한 작업을 콜레이터가 알아서 자동으로 처리한다.

Metric
마지막으로 학습한 모델을 측정할 매트릭을 준비해야 한다. 번역 모델에서는 주로 BLEU 점수를 사용한다. BLEU 점수는 번역기가 생성한 문장이 레퍼런스(정답이라는 표현을 사용하지 않는 이유는 제대로 된 번역 문장이 오직 하나가 아니기 때문)문장과 얼마나 비슷한지 측정하는 점수라고 생각하면 된다. 단 같은 단어가 반복된다든지 레퍼런스 문장보다 너무 짧은 문장을 생성한다든지 하면 패널티를 부여 한다. 그렇기 때문에 레퍼런스 문장과 길이가 최대한 비슷하고 다양한 단어를 사용하면서 생성된 문장의 단어가 레퍼런스 단어에 많이 보여야 높은 점수를 얻게 된다.

BLEU를 계산하기 위해 허깅페이스 evaluate 라이브러리와 sacrebleu라이브러리를 제일 처음에 설치했었다.

sacrebleu 라이브러리는 BLEU 구현체에서 사실상 표준 라이브러리이며 각 모델이 다른 토크나이저를 쓰는 경우 이를 BPE로 통일 시켜 BLEU 점수를 계산한다고 한다. 참고링크

evaluate라이브러리로 이 sacrebleu를 불러온다.

In [51]:
import evaluate

metric = evaluate.load("sacrebleu")

In [52]:
predictions = [
    "저는 딥러닝을 좋아해요.",
    "요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다."
]

references = [
    ["저는 딥러닝을 좋아해요.", "나는 딥러닝을 사랑해요."],
    ["요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다.",
     "최근에는 딥러닝 프레임워크가 잘 개발되어 있기 때문에 다른 사람의 도움 없이도 기계 번역 시스템을 개발할 수 있습니다."]
]
metric.compute(predictions=predictions, references=references)

{'score': 100.00000000000004,
 'counts': [21, 19, 17, 15],
 'totals': [21, 19, 17, 15],
 'precisions': [100.0, 100.0, 100.0, 100.0],
 'bp': 1.0,
 'sys_len': 21,
 'ref_len': 21}

In [53]:
predictions = [
    "저는 딥러닝을 좋아해요.",
    "딥러닝 프레임워크가 잘 개발되었기 때문에 요즘은 누군가의 도움 없이 기계번역 시스템을 구축할 수 있다."
]

references = [
    ["저는 딥러닝을 좋아해요.", "나는 딥러닝을 사랑해요."],
    ["요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다.",
     "최근에는 딥러닝 프레임워크가 잘 개발되어 있기 때문에 다른 사람의 도움 없이도 기계 번역 시스템을 개발할 수 있습니다."]
]
metric.compute(predictions=predictions, references=references)

{'score': 25.28116160010779,
 'counts': [14, 7, 4, 1],
 'totals': [19, 17, 15, 13],
 'precisions': [73.6842105263158,
  41.1764705882353,
  26.666666666666668,
  7.6923076923076925],
 'bp': 0.9000876262522591,
 'sys_len': 19,
 'ref_len': 21}

점수가 떨어지는 것을 확인할 수 있다. 아래 함수는 모델의 예측과 레이블을 가지고 bleu를 계산하는 헬퍼 함수로 트랜스포머 학습 코스 번역기 매트릭에서 제공하는 코드를 그대로 복사 한 것이다.

In [54]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result

Trainer
학습을 간단히 하기위해 허깅페이스에서 제공하는 Seq2SeqTrainer클래스를 사용한다. 학습 세부 조건은 Seq2SeqTrainingArguments를 사용하여 설정한다. 다음 코드로 학습에 필요한 세부 사항을 설정할 수 있다.

In [55]:
fine_tuned_model_checkpoint = "ke_t5_base_bongsoo_en_ko"
output_dir = fine_tuned_model_checkpoint
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    learning_rate=0.0005,
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_strategy="no",
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    push_to_hub=True, # Wandb 로그 끄기
)

이런 저런 자잘한 세팅을 해서 training_args를 만들고 trainer를 생성한다. 지금까지 준비한 model, training_args, tokenized_datasets, data_collator, tokenizer, compute_metrics를 넘기면 된다.

In [56]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,


In [57]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,No log,2.407457,8.227161


TrainOutput(global_step=5625, training_loss=3.63875, metrics={'train_runtime': 12750.4853, 'train_samples_per_second': 28.234, 'train_steps_per_second': 0.441, 'total_flos': 2.70316407619584e+16, 'train_loss': 3.63875, 'epoch': 1.0})

In [58]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

'https://huggingface.co/chunwoolee0/ke_t5_base_bongsoo_en_ko/tree/main/'

In [59]:
model_dir = "chunwoolee0/ke_t5_base_bongsoo_en_ko"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu();

In [60]:
input_text = [
    "Because deep learning frameworks are well developed, in these days, machine translation system can be built without anyone's help.",
    "This system was made by using HuggingFace's T5 model for a one day"
]

In [61]:
inputs = tokenizer(input_text, return_tensors="pt",
                   padding=True, max_length=max_token_length)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


inputs를 확인해보면 input_ids와 attention_mask로 토큰화 된것을 알 수 있다. 첫번째 문장이 더 길기 때문에 두번째 문장의 마스크는 마지막에 0으로 패딩된 것도 확인할 수 있다.

In [62]:
inputs

{'input_ids': tensor([[ 8127,  5859,  5789, 22309,     8,    69,   484,  6560,     4,    20,
           572,  1258,     4,  9872, 46301,  1076,   147,    67,  3807,  1215,
          3993,    17,     8,   787,     3,     1],
        [  465,  1076,    62,   565,    81,  1676,   992, 60049,  1044, 17400,
            17,     8,   745,   466,  3900,    40,    16,   165,   688,     1,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0]])}

In [63]:
koreans = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

koreans.shape

torch.Size([2, 21])

In [64]:
[
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(korean)) for korean in koreans
]

['<pad> 딥러닝 틀이 잘 개발되기 때문에 요즘은 누구도 도움을 받지 않고도 기계번역 시스템을 만들 수 있다.</s>',
 '<pad> 이 시스템은 챗페이의 T5 모델을 하루 동안 이용해 만든 시스템이다.</s><pad><pad><pad><pad>']

마지막으로 테스트 셋에 대해서 몇개 문장을 가져와 번역해보자. 만들어 놓은 tokenized_datasets과 data_collator를 pytorch DataLoader에 그대로 전달해서 데이터 로더를 만들 수 있다.

In [65]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=32, collate_fn=data_collator
)

In [66]:
test_dataloader_iter = iter(test_dataloader)

In [67]:
test_batch = next(test_dataloader_iter)

콜레이터에 의해 반환된 미니 배치에는 다음처럼 labels, decoder_input_ids 따위도 가지고 있으므로 모델에 입력하기 위해 input_ids, attention_mask만 남긴다.

In [68]:
test_batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [69]:
test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }

In [70]:
koreans = model.generate(
    **test_input,
    max_length=max_token_length,
    num_beams=5,
)

이제 입력문장, 정답 그리고 생성된 문장을 비교하기 위해 우선 test_batch.labels에 -100으로 인코딩된 부분을 패딩 코튼으로 교체 한다.

In [71]:
labels =  np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)

In [72]:
eng_sents = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True)[10:20]

In [73]:
references = tokenizer.batch_decode(labels, skip_special_tokens=True)[10:20]

In [74]:
preds = tokenizer.batch_decode( koreans, skip_special_tokens=True )[10:20]

In [75]:
for s in zip(eng_sents, references, preds):
    print('English   :', s[0])
    print('Reference :', s[1])
    print('Translated:', s[2])
    print('\n')

English   : Researcher Heo was in charge of operating Korea Immigration & Integration Program (KIIP) of the Ministry of Justice since 2012, providing basic education needed for immigrants to adapt as members of Korean society such as the Korean language, Korean culture, and understanding of Korean society.
Reference : 허 연구원은 2012년부터 법무부 사회통합프로그램(KIIP·Korea Immigration & Integration Program) 운영을 맡아 이민자가 우리사회 구성원으로 적응하는데 필요한 한국어, 한국문화, 한국사회 이해 등 기본소양 교육을 진행해왔다.
Translated: 허 연구원은 2012년부터 법무부의 한국 이민·통합 프로그램(KIIP) 운영을 담당해 탈북민이 한국어, 한국 문화, 한국 사회 이해 등 한국 사회 구성원으로 적응하기 위해 필요한 기초 교육을 제공했다.


English   : Under this situation, the observation has also been raised that Trump administration who are currently having a pro-Israel policy will even give the West Bank settlement to Israel.
Reference : 이런 상황에서 친이스라엘 정책을 펴고 있는 트럼프 행정부가 요르단강 서안 정착촌마저 이스라엘에 내줄 것이라는 관측도 제기된다.
Translated: 이런 상황에서 현재 친이스라엘 정책을 펴고 있는 트럼프 행정부가 이스라엘에 웨스트뱅크 정착까지 내주겠다는 관측도 제기됐다.


English   : Many good social science books are ava